<a href="https://colab.research.google.com/github/Nick088Official/GFPGAN-Fix/blob/master/GFPGAN_fix_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GFPGAN Inference Demo  (Restore faces of images)

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2101.04061)
[![GitHub Stars](https://img.shields.io/github/stars/TencentARC/GFPGAN?style=social)](https://github.com/TencentARC/GFPGAN)
[![download](https://img.shields.io/github/downloads/TencentARC/GFPGAN/total.svg)](https://github.com/TencentARC/GFPGAN/releases)

## GFPGAN - Towards Real-World Blind Face Restoration with Generative Facial Prior

GFPGAN is a blind face restoration algorithm towards real-world face images. <br>
It leverages the generative face prior in a pre-trained GAN (*e.g.*, StyleGAN2) to restore realistic faces while precerving fidelity. <br>

**Limitations**: GFPGAN could not handle all the low-quality faces in the real world. Therefore, it may fail on your own cases.

###Enjoy! :-)

<img src="https://xinntao.github.io/projects/GFPGAN_src/gfpgan_teaser.jpg" width="800">


In [ ]:
#@title Installation
#@markdown Before start, make sure that you choose
#@markdown * Runtime Type = Python 3
#@markdown * Hardware Accelerator = GPU

#@markdown in the **Runtime** menu -> **Change runtime type**

#@markdown By running this, we clone the repository, set up the envrironment, and download the pre-trained model which we use 1.2 , 1.3 , 1.4 , RestoreFormer , CodeFormer.

# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN-Fix
!git clone https://github.com/Nick088Official/GFPGAN-Fix.git
%cd GFPGAN-Fix

import torch
from IPython.display import clear_output
import numpy as np
import glob
from os.path import isfile, join
import os
from google.colab import files
import shutil

if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

# Set up the environment
# We install and use new-BasicSR for both training and inference
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
!pip uninstall -y basicsr # have to uninstall it as it gives error to GFPGAN but still gets installed from other package
!pip install -r requirements.txt # which installs other packages such as new-basicsr

if device == "cuda":
  !BASICSR_EXT=True pip install new-basicsr # for installing cuda extensions for v1 model for colorization

# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.1.0/GFPGANv1.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/RestoreFormer.pth -P experiments/pretrained_models
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/CodeFormer.pth -P experiments/pretrained_models
clear_output()
print("Installed GFPGAN, its requirements and the models.")

In [ ]:
#@title Upload Input Image

upload_folder = 'inputs/upload'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)

# upload input
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'moved {filename} to {dst_path}')
  shutil.move(filename, dst_path)

## 3. Inference
Usage: `python inference_gfpgan.py -i inputs/whole_imgs -o results [options]...`

Options:
```
  -v version           GFPGAN model version. Option: 1 | 1.2 | 1.3 | 1.4. | RestoreFormer
  -s upscale           The final upsampling scale of the image. Default: 2
  -bg_upsampler        background upsampler. Default: realesrgan
  -bg_tile             Tile size for background sampler, 0 for no tile during testing. Default: 400
  -suffix              Suffix of the restored faces
  -only_center_face    Only restore the center face
  -aligned             Input are aligned faces
  -ext                 Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto
```

In [ ]:
#@title Inference

!rm -rf results

options = "-v 1.3 -s 2 --bg_upsampler realesrgan" #@param {type:"string"}

!python inference_gfpgan.py -i inputs/upload -o results {options}

!ls results/cmp

In [ ]:
#@title Visualize Input vs Output Faces
# We first visualize the cropped faces
# The left are the inputs images; the right are the results of GFPGAN

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('GFPGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_folder = 'results/cropped_faces'
result_folder = 'results/restored_faces'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)


In [ ]:
#@title View the whole Input vs Output Image
# We then visualize the whole image
# The left are the inputs images; the right are the results of GFPGAN

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('GFPGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

input_folder = 'inputs/upload'
result_folder = 'results/restored_imgs'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)

In [ ]:
#@title Download results
#@markdown It will download you a .zip, unzip it however you want
#@markdown - The 'cmp' folder contains a comparison between the Input vs Output faces,
#@markdown - The 'cropped_faces' folder contains only the cropped Input Face
#@markdown - The 'restored_faces' folder contains only the Output Face (Restored one)
#@markdown - The 'restored_imgs' folder contains the whole Output Image restored with also the Background

# download the result
!ls results
print('Download results')
os.system('zip -r download.zip results')
files.download("download.zip")